In [1]:
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm

In [11]:
from evaluating_file import *
sys.path.append('../../datasets/Time_Series_Segmentation_Benchmark')
from utils import *
sys.path.append('../../benchmark/utils')
# from evaluation import *
sys.path.append('../')
from ClassificationParScoreProfile import *
from BinarySegmentation import *
from DynamicProgramming import *
from FLOSS import *
from Pelt import *
from Window import *
from BayesianOnlineChangePointDetection import *
from Fluss import *

<h3>Load Dataset</h3>

In [3]:
tssb = load_time_series_segmentation_datasets()

In [4]:
tssb

,dataset,window_size,change_points,time_series
0,Adiac,10,"[572, 1012, 1232]","[1.681146, 1.383394, 1.029389, 0.688386, 0.354..."
1,ArrowHead,10,[753],"[-1.957721, -1.858082, -1.657983, -1.420794, -..."
2,Beef,50,[705],"[-0.380818, -0.352238, -0.295392, -0.225041, -..."
3,BeetleFly,10,[1280],"[1.114805, 0.774403, 0.477129, 0.115781, -0.24..."
4,BirdChicken,20,[1280],"[1.697714, 1.439032, 1.22287, 0.99708, 0.76479..."
...,...,...,...,...
70,UWaveGestureLibraryX,10,"[600, 1131, 1652, 2193]","[-0.889766, 0.722005, 0.869297, -0.553167, -0...."
71,UWaveGestureLibraryY,10,"[600, 1131, 1652, 2193]","[-1.523467, -0.106794, 0.190361, 0.979247, 0.3..."
72,UWaveGestureLibraryZ,10,"[600, 1131, 1652, 2193]","[-0.527058, 0.416111, -0.524027, 0.41141, 0.12..."
73,WordSynonyms,10,"[202, 2227]","[-1.183581, -0.514028, -0.480539, -0.394855, -..."


<h3>Evaluate Algorithms</h3>

In [5]:
results={}
def evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,algorithm_name):
    # Initialize lists to store metric values for each time series
    covering_scores = []
    f_measure_scores = []
    nab_scores = []
    i=0
    # Iterate over each time series
    for cps_true,cps_found,ts_len in zip(all_cps_true,all_cps_found,all_ts_len):
        i=i+1
        score_covering = covering({0: cps_true}, cps_found, ts_len)
        covering_scores.append(score_covering)
        
        # Calculate F-measure
        score_F = f_measure({0: cps_true}, cps_found, ts_len) 
        f_measure_scores.append(score_F)
        
        # Format true and predicted change points as Series
        true_cp, predicted_cp = formate_data(cps_true, cps_found,ts_len)
    
        # Evaluate change points using the evaluating_change_point function
        score_NAB = evaluating_change_point(true_cp, predicted_cp, metric='nab')
        nab_scores.append(score_NAB)
        # print(score_NAB)

    # Calculate mean scores across all time series
    mean_covering = sum(covering_scores) / len(covering_scores)
    mean_f_measure = sum(f_measure_scores) / len(f_measure_scores)

    # Calculate mean NAB scores
    nab_scores_filtered = [score for score in nab_scores if not np.isnan(score['Standart']) and not np.isinf(score['Standart'])]
    mean_nab_standard = sum(score['Standart'] for score in nab_scores_filtered) / len(nab_scores_filtered)
    mean_nab_lowFP = sum(score['LowFP'] for score in nab_scores_filtered) / len(nab_scores_filtered)
    mean_nab_lowFN = sum(score['LowFN'] for score in nab_scores_filtered) / len(nab_scores_filtered)


    # Print mean scores
    print(f"Mean Covering: {mean_covering}")
    print(f"Mean F-measure: {mean_f_measure}")
    print(f"Mean NAB (Standard): {mean_nab_standard}")
    print(f"Mean NAB (LowFP): {mean_nab_lowFP}")
    print(f"Mean NAB (LowFN): {mean_nab_lowFN}")

    results[algorithm_name]={"Covering":mean_covering,
                                        "F-measure":mean_f_measure,
                                        "NAB (Standard)":mean_nab_standard,
                                        "NAB (LowFP)":mean_nab_lowFP,
                                        "NAB (LowFN)":mean_nab_lowFN
    }


<h3>BinaryClaSPSegmentation</h3>

In [6]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
# Iterate over each time series
for _, (ts_name, window_size, cps_true, ts) in tqdm(tssb.iterrows()):
    # Fit the BinaryClaSPSegmentation algorithm and predict change points
    cps_found = BinaryClaSPSegmentation().fit_predict(ts)
    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"BinaryClaSPSegmentation")

0it [00:00, ?it/s]

75it [03:54,  3.12s/it]
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarni

Mean Covering: 0.8547707161202618
Mean F-measure: 0.8955997727762428
Mean NAB (Standard): 49.383043478260866
Mean NAB (LowFP): 43.30811594202898
Mean NAB (LowFN): 51.57260869565218


<h3>DynamicProgramming</h3>

In [7]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
# Iterate over each time series
for _, (ts_name, window_size, cps_true, ts) in tqdm(tssb.iterrows()):
    # Fit the DynamicProgramming algorithm and predict change points
    cps_found = DynamicProgramming().fit_predict(np.array(ts), len(cps_true))[:-1]
    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"DynamicProgramming")

75it [1:17:11, 61.75s/it] 
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWa

Mean Covering: 0.46785517407284694
Mean F-measure: 1.0
Mean NAB (Standard): 29.161884057971005
Mean NAB (LowFP): 23.32536231884058
Mean NAB (LowFN): 32.52753623188404


<h3>FLOSS</h3>

In [11]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
# Iterate over each time series
for _, (ts_name, window_size, cps_true, ts) in tqdm(tssb.iterrows()):
    floss_detector = FLOSS(n_timepoints=len(ts), window_size=20, n_prerun=500, threshold=0.7, excl_factor=5, verbose=0)
    # Update FLOSS with each time point
    for timepoint in ts:
        floss_detector.update(timepoint)

    # Get the change points detected by FLOSS
    cps_found = floss_detector.change_points
    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"FLOSS")


75it [18:26, 14.75s/it]
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: divide by zero encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: divide by zero encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: divide by zero encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWa

Mean Covering: 0.5193691811281659
Mean F-measure: 0.5146524852335557
Mean NAB (Standard): 51.01507246376811
Mean NAB (LowFP): 19.708115942028986
Mean NAB (LowFN): 61.46652173913044


<h3>Pelt</h3>

In [17]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
# Iterate over each time series
for _, (ts_name, window_size, cps_true, ts) in tqdm(tssb.iterrows()):
     # Predict change points using the Pelt algorithm&
    cps_found = Pelt().fit_predict(np.array(ts), pen=45)[:-1]
    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"Pelt")


d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: divide by zero encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encou

Mean Covering: 0.3759451685693726
Mean F-measure: 0.4834183328866255
Mean NAB (Standard): 2.68768115942029
Mean NAB (LowFP): -2.9402898550724643
Mean NAB (LowFN): 4.690579710144928


d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: divide by zero encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)


<h3>BOCD</h3>

In [19]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
# Iterate over each time series
for _, (ts_name, window_size, cps_true, ts) in tqdm(tssb.iterrows()):
    # Fit the BinaryClaSPSegmentation algorithm and predict change points
    cps_found = bocd(ts,len(cps_true))
    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"BOCD")

d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encoun

Mean Covering: 0.5455411250579468
Mean F-measure: 1.0
Mean NAB (Standard): 27.403478260869555
Mean NAB (LowFP): 21.974637681159415
Mean NAB (LowFN): 29.386666666666656


d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)


<h3>Fluss</h3>

In [19]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
# Iterate over each time series
for _, (ts_name, window_size, cps_true, ts) in tqdm(tssb.iterrows()):
    # Parameters
    window_size = 50
    n_cps = 3
    # Using FLUSS function to predict change points
    cps_found = fluss(ts, window_size, n_cps)

    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"Fluss")



75it [03:28,  2.77s/it]
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: divide by zero encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: divide by zero encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: divide by zero encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeWarning: divide by zero encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\evaluating_file.py:379: RuntimeW

Mean Covering: 0.6661554089503763
Mean F-measure: 0.7953333333333329
Mean NAB (Standard): 40.27768115942028
Mean NAB (LowFP): 34.56942028985507
Mean NAB (LowFN): 42.38014492753623


In [20]:
# Convert dictionary to DataFrame
results_df = pd.DataFrame(results).T.reset_index()

# Rename the 'index' column to 'name_algo'
results_df = results_df.rename(columns={'index': 'name_algo'})
# save the file
results_df.to_csv('results/TSSB.csv', index=False)

print("DataFrame saved as 'TSSB.csv'")
# Display the DataFrame
results_df

DataFrame saved as 'TSSB.csv'


,name_algo,Covering,F-measure,NAB (Standard),NAB (LowFP),NAB (LowFN)
0,BinaryClaSPSegmentation,0.854771,0.895600,49.383043,43.308116,51.572609
1,DynamicProgramming,0.467855,1.000000,29.161884,23.325362,32.527536
2,FLOSS,0.519369,0.514652,51.015072,19.708116,61.466522
3,Pelt,0.375945,0.483418,2.687681,-2.940290,4.690580
4,Window,0.401032,0.512508,0.000000,0.000000,0.000000
5,Fluss,0.666155,0.795333,40.277681,34.569420,42.380145
